<a href="https://colab.research.google.com/github/asyhar22/webscrap/blob/main/Scrapping_Data_Tour_de_France.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Scrapping Data Tour de France tahun 2010 sd 2022**
by Muhammad Asyhar

Sebelum memulai, perlu dilakukan inspect element di browser untuk mengetahui skema pemanggilan data melalui web, dalam hal ini misalnya untuk memperoleh data Tahun 2022, Stage 21, Tipe Individual(Stage) adalah dengan memanggil https://letour.fr/en/block/history/11822/ca646e1b4b99d7ee973c457c87392395?stage=21&type=ite di mana
- URL = "https://letour.fr"
- Path = "/en/block/history/11822/ca646e1b4b99d7ee973c457c87392395" --> mewakili Tahun
- Parameter = "stage=21&type=ite" --> mewakili Stage dan Type Rangking

## Langkah langkah:
1. Mengakses halaman https://www.letour.fr/en/history
2. Mendapatkan path header untuk masing-masing tahun
3. Mendefinisikan race type
4. Mendefinisikan stage
5. Membuat URL dengan menggabungkan data pada langkah 2, 3, dan 4
6. Membuat dataframe dari tabel response (sekaligus menambahkan kolom tahun dan stage, serta membuat race id eg:202201 (tahun 2022 stage 1))
7. Menyimpan file ke csv

####1. Mengakses halaman https://www.letour.fr/en/history

In [1]:
import requests

url = 'https://www.letour.fr/en/history'
r = requests.get(url)
r.status_code

200

status code 200 menunjukan bahwa akses ke URL tersebut berhasil

####2. Mendapatkan path header untuk masing-masing tahun

In [2]:
from bs4 import BeautifulSoup

In [3]:
soup = BeautifulSoup(r.text,'html.parser')

In [4]:
#hasil inspect browser
year_test = 2010
path_test = soup.find('button',{'class': 'dateTabs__link js-tabs','data-tabs-target':year_test})
path_test

<button class="dateTabs__link js-tabs" data-clicktype="A" data-tabs-ajax="/en/block/history/10803/757e8e81b7183fde7329fd8ab9b69783" data-tabs-target="2010" data-xtclick="history::navTabs::item::link::2010">2010</button>

Langkah ini bertujuan untuk memperoleh bagian "/en/block/history/10803/757e8e81b7183fde7329fd8ab9b69783" yang merupakan bagian unik yang mewakili data "Tahun".

In [5]:
#testing untuk menguji code yang tepat untuk memanggil path
print(path_test['data-tabs-ajax'])

/en/block/history/10803/757e8e81b7183fde7329fd8ab9b69783


In [6]:
#membuat list tahun yang akan diambil datanya
years = list(range(2010,2023))
years

[2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]

In [7]:
path = list(soup.find('button',{'class': 'dateTabs__link js-tabs','data-tabs-target':year})['data-tabs-ajax'] for year in years)
path

['/en/block/history/10803/757e8e81b7183fde7329fd8ab9b69783',
 '/en/block/history/10804/89b36b01ffe439e016ec1d59c57b63d1',
 '/en/block/history/10805/1ec24b4375d16b922c9c9c489d25465c',
 '/en/block/history/10806/196e36cfe7aff4b5fc2d1055f7dcf864',
 '/en/block/history/10807/d538a7fbdfc0d657fbf064561840fbb1',
 '/en/block/history/10808/3f166bedb535ee9bff2fb7ead0a7812c',
 '/en/block/history/10809/7be3a459d846b4672915c576cb7ed6b9',
 '/en/block/history/10810/7035a5dc53631209d3581f64a433b10d',
 '/en/block/history/11818/f34c3404d95a697dcf77d4cd8e8278fa',
 '/en/block/history/11819/96c0eb3fa403ebf222f28b6e45115c56',
 '/en/block/history/11820/17fa8e795e69e9f326ea26cf9912e571',
 '/en/block/history/11821/a3fe7d350ef546e3c27ae83a84fa3a0b',
 '/en/block/history/11822/ca646e1b4b99d7ee973c457c87392395']

path untuk masing-masing tahun sudah diperoleh, selanjutnya perlu dibuat dictionary agar masing-masing path dapat teridentifikasi pada saat dilakukan otomasi kemudian.

In [8]:
path_dict = dict(zip(years,path))
path_dict

{2010: '/en/block/history/10803/757e8e81b7183fde7329fd8ab9b69783',
 2011: '/en/block/history/10804/89b36b01ffe439e016ec1d59c57b63d1',
 2012: '/en/block/history/10805/1ec24b4375d16b922c9c9c489d25465c',
 2013: '/en/block/history/10806/196e36cfe7aff4b5fc2d1055f7dcf864',
 2014: '/en/block/history/10807/d538a7fbdfc0d657fbf064561840fbb1',
 2015: '/en/block/history/10808/3f166bedb535ee9bff2fb7ead0a7812c',
 2016: '/en/block/history/10809/7be3a459d846b4672915c576cb7ed6b9',
 2017: '/en/block/history/10810/7035a5dc53631209d3581f64a433b10d',
 2018: '/en/block/history/11818/f34c3404d95a697dcf77d4cd8e8278fa',
 2019: '/en/block/history/11819/96c0eb3fa403ebf222f28b6e45115c56',
 2020: '/en/block/history/11820/17fa8e795e69e9f326ea26cf9912e571',
 2021: '/en/block/history/11821/a3fe7d350ef546e3c27ae83a84fa3a0b',
 2022: '/en/block/history/11822/ca646e1b4b99d7ee973c457c87392395'}

####3. Mendefinisikan race type

In [9]:
#untuk individual (stage)
race_type = 'ite'

####4. Mendefinisikan stage

In [10]:
#ada 21 stage dalam setiap gelaran TDF
stage_list = list(range(1,22))
stage_list

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]

####5. Membuat URL request untuk masing-masing tahun, race type, & stage

In [11]:
# untuk masing masing tahun
for year in years:
  # untuk masing-masing stage
  for stage in stage_list:
    print(f'https://letour.fr{path_dict[year]}?stage={stage}&type={race_type}')

https://letour.fr/en/block/history/10803/757e8e81b7183fde7329fd8ab9b69783?stage=1&type=ite
https://letour.fr/en/block/history/10803/757e8e81b7183fde7329fd8ab9b69783?stage=2&type=ite
https://letour.fr/en/block/history/10803/757e8e81b7183fde7329fd8ab9b69783?stage=3&type=ite
https://letour.fr/en/block/history/10803/757e8e81b7183fde7329fd8ab9b69783?stage=4&type=ite
https://letour.fr/en/block/history/10803/757e8e81b7183fde7329fd8ab9b69783?stage=5&type=ite
https://letour.fr/en/block/history/10803/757e8e81b7183fde7329fd8ab9b69783?stage=6&type=ite
https://letour.fr/en/block/history/10803/757e8e81b7183fde7329fd8ab9b69783?stage=7&type=ite
https://letour.fr/en/block/history/10803/757e8e81b7183fde7329fd8ab9b69783?stage=8&type=ite
https://letour.fr/en/block/history/10803/757e8e81b7183fde7329fd8ab9b69783?stage=9&type=ite
https://letour.fr/en/block/history/10803/757e8e81b7183fde7329fd8ab9b69783?stage=10&type=ite
https://letour.fr/en/block/history/10803/757e8e81b7183fde7329fd8ab9b69783?stage=11&type=i

####6. Membuat dataframe dari tabel response
(sekaligus menambahkan kolom tahun dan stage,
serta membuat race id eg:202201 (tahun 2022 stage 1))

In [12]:
import pandas as pd

In [13]:
# testing sebelum memulai otomasi untuk meninjau kode dan hasil
html_table = pd.read_html('https://letour.fr/en/block/history/11822/ca646e1b4b99d7ee973c457c87392395?stage=21&type=ite')
df = pd.DataFrame(html_table[0])
df['Rider No.'] = df['Rider No.'].apply("{0:0=3d}".format, axis=1)
df['Year'] = 2022
df['Stage'] = 1
df['Race_Id'] = df['Year'].astype(str) + df['Stage'].astype(str)
df['Rider_Id'] = df['Year'].astype(str) + df['Rider No.']
df.tail()

,Rank,Rider,Rider No.,Team,Times,Gap,B,P,Year,Stage,Race_Id,Rider_Id
130,131,FREDERIK FRISON,162,LOTTO SOUDAL,85h 03' 39'',+ 05h 30' 19'',NaN,NaN,2022,1,20221,2022162
131,132,REINARDT JANSE VAN RENSBURG,164,LOTTO SOUDAL,85h 04' 45'',+ 05h 31' 25'',NaN,NaN,2022,1,20221,2022164
132,133,AMUND GRÃNDAHL JANSEN,205,TEAM BIKEEXCHANGE-JAYCO,85h 04' 47'',+ 05h 31' 27'',NaN,NaN,2022,1,20221,2022205
133,134,ALBERT TORRES BARCELO,067,MOVISTAR TEAM,85h 09' 53'',+ 05h 36' 33'',NaN,NaN,2022,1,20221,2022067
134,135,CALEB EWAN,161,LOTTO SOUDAL,85h 14' 02'',+ 05h 40' 42'',NaN,01' 00'',2022,1,20221,2022161


setelah kode berhasil dijalankan dan hasilnya sudah sesuai, selanjutnyalangkah  adalah membuat dataframe dengan teknik otomasi sehingga dapat terbentuk sebuah dataframe yang mencakup semua stage dari tahu 2010 sd 2022 ditambah dengan kolom tambahan (year, stage, race_id) untuk membedakan data dari masing-masing tahun.

In [14]:
# membuat df kosong untuk menampung hasil otomasi
df = pd.DataFrame()

# membuat iterasi dengan memodifikasi kode pada langkah ke-5 agar tidak hanya menampilkan url tetapi juga membuat request
for year in years:
  for stage in stage_list:
    url = f'https://letour.fr{path_dict[year]}?stage={stage}&type={race_type}'
    html_table = pd.read_html(url)
    df_to_append = pd.DataFrame(html_table[0])
    df_to_append['Rider No.'] = df_to_append['Rider No.'].apply("{0:0=3d}".format, axis=1)
    df_to_append['Year'] = str(year)
    df_to_append['Stage'] = "{0:0=2d}".format(stage)
    df_to_append['Race_Id'] = df_to_append['Year'] + df_to_append['Stage']
    df_to_append['Rider_Id'] = df_to_append['Year'] + df_to_append['Rider No.']
    df = pd.concat([df, df_to_append], ignore_index=True)


####7. Menyimpan file ke csv

In [15]:
#preview
df.head()

,Rank,Rider,Rider No.,Team,Times,Gap,B,P,Year,Stage,Race_Id,Rider_Id
0,1,ANDY SCHLECK,011,TEAM SAXO BANK,91h 59' 27'',-,NaN,NaN,2010,01,201001,2010011
1,2,SAMUEL SANCHEZ,181,EUSKALTEL - EUSKADI,92h 02' 28'',+ 00h 03' 01'',NaN,NaN,2010,01,201001,2010181
2,3,JURGEN VAN DEN BROECK,101,OMEGA PHARMA - LOTTO,92h 05' 42'',+ 00h 06' 15'',NaN,NaN,2010,01,201001,2010101
3,5,ROBERT GESINK,195,RABOBANK,92h 08' 19'',+ 00h 08' 52'',NaN,NaN,2010,01,201001,2010195
4,6,RYDER HESJEDAL,054,GARMIN - TRANSITIONS,92h 09' 03'',+ 00h 09' 36'',NaN,NaN,2010,01,201001,2010054


In [16]:
df.to_csv("TDF Individual Stage 2010-2022.csv", index=False)

Demikian proses scraping atau pengambilan data dari website ajang pada balap sepeda tour de france dari tahun 2010 sd 2022. Pengambilan data pada notebook ini dibatasi pada rangking individual per stage, sementara masih banyak data lain dari situs tersebut yang masih bisa dieksplorasi lebih lanjut. Notebook ini bisa digunakan untuk melakukan eksplorasi tersebut, tentunya dengan beberapa penyesuaian.

# **Terima kasih**